# Setting up and Testing an Agent for Amazon Bedrock with Code Interpreter

In this notebook, we'll walk through the process of creating, testing, and cleaning up an Agent in Amazon Bedrock. We'll see how to set up the Code Interpreter action.  Code Interpreter enables your agent to write and execute code, process documents, and respond to complex queries via access to a secure code execution sandbox.

Let's dive in!

_(Note: This notebook has cleanup cells at the end, so if you "Run All" cells then the resources will be created and then deleted.)_

_Note: At the time of writing Code Interpreter is in public preview.  

## Step 1: Import Required Libraries

First, we need to import the necessary Python libraries. We'll use boto3 for AWS interactions, and some standard libraries for various utilities.

In [ ]:
import boto3
import json
import time, random 
import uuid, string
import matplotlib.pyplot as plt
import io

## Step 2: Set the AWS Region

We're using the US East (N. Virginia) region for this demo. Feel free to change this to your preferred region, but make sure that a) the region supports Amazon Bedrock, b) Agents, c) the Claude Sonnet (3) model, and finally d) you have enabled access to the Sonnet (3) in this region. 

In [ ]:
region_name = 'us-east-1'

## Step 3: Create the Bedrock Agent

Now comes the exciting part! We're going to set up our Bedrock Agent. This involves creating an IAM role, setting up policies, and configuring the agent itself. We'll use Claude 3 Sonnet as our foundation model.

In [ ]:
# Set up Bedrock Agent and IAM clients
bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)
iam = boto3.client('iam')

agentName = 'code-interpreter-test-agent'

# Define the agent's personality and behavior
instruction = """You are an advanced AI agent with capabilities in code execution, chart generation, and complex data analysis. Your primary function is to assist users by solving problems and fulfilling requests through these capabilities. Here are your key attributes and instructions:

Code Execution:

You have access to a Python environment where you can write and execute code in real-time.
When asked to perform calculations or data manipulations, always use this code execution capability to ensure accuracy.
After executing code, report the exact output and explain the results.


Data Analysis:

You excel at complex data analysis tasks. This includes statistical analysis, data visualization, and machine learning applications.
Approach data analysis tasks systematically: understand the problem, prepare the data, perform the analysis, and interpret the results.


Problem-Solving Approach:

When presented with a problem or request, break it down into steps.
Clearly communicate your thought process and the steps you're taking.
If a task requires multiple steps or tools, outline your approach before beginning.


Transparency and Accuracy:

Always be clear about what you're doing. If you're running code, say so. If you're generating an image, explain that.
If you're unsure about something or if a task is beyond your capabilities, communicate this clearly.
Do not present hypothetical results as actual outcomes. Only report real results from your code execution or image generation.


Interaction Style:

Be concise in simple queries but provide detailed explanations for complex tasks.
Use technical language appropriately, but be prepared to explain concepts in simpler terms if asked.
Proactively offer relevant information or alternative approaches that might be helpful.


Continuous Improvement:

After completing a task, ask if the user needs any clarification or has follow-up questions.
Be receptive to feedback and adjust your approach accordingly.


Remember, your goal is to provide accurate, helpful, and insightful assistance by leveraging your unique capabilities in code execution, image generation, and data analysis. Always strive to give the most practical and effective solution to the user's request."""

# Specify the foundation model to use (for 'code interpreter' it must be Sonnet or Haiku).
foundationModel = 'anthropic.claude-3-sonnet-20240229-v1:0'

# Generate a random suffix for unique naming
randomSuffix = "".join(
    random.choices(string.ascii_uppercase + string.digits, k=5)
)

print("Creating the IAM policy and role...")

# Define IAM trust policy
trustPolicy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Define IAM policy for invoking the foundation model
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel"
            ],
            "Resource": [
                f"arn:aws:bedrock:{region_name}::foundation-model/{foundationModel}"
            ]
        }
    ]
}

role_name = f"test-agent-{randomSuffix}"

# Create IAM role and attach policy
role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument = json.dumps(trustPolicy)
)
iam.put_role_policy(
    RoleName=role_name,
    PolicyName = f"policy-test-agent-{randomSuffix}",
    PolicyDocument = json.dumps(policy)
)

roleArn = role['Role']['Arn']

print(f"IAM Role: {roleArn[:13]}{'*' * 12}{roleArn[25:]}")

print("Creating the agent...")

# Create the Bedrock Agent
response = bedrock_agent.create_agent(
    agentName=f"{agentName}-{randomSuffix}",
    foundationModel=foundationModel,
    instruction=instruction,
    agentResourceRoleArn=roleArn,
)

agentId = response['agent']['agentId']

print("Waiting for agent status of 'NOT_PREPARED'...")

# Wait for agent to reach 'NOT_PREPARED' status
agentStatus = ''
while agentStatus != 'NOT_PREPARED':
    response = bedrock_agent.get_agent(
        agentId=agentId
    )
    agentStatus = response['agent']['agentStatus']
    print(f"Agent status: {agentStatus}")
    time.sleep(2)

######################################### Configure code interpreter for the agent
response = bedrock_agent.create_agent_action_group(
    
    actionGroupName='CodeInterpreterAction',
    actionGroupState='ENABLED',
    agentId=agentId,
    agentVersion='DRAFT',

    parentActionGroupSignature='AMAZON.CodeInterpreter' # <-  To allow your agent to generate, 
                                                        #     run, and troubleshoot code when trying 
                                                        #     to complete a task, set this field to 
                                                        #     AMAZON.CodeInterpreter. 
                                                        #     You must leave the `description`, `apiSchema`, 
                                                        #     and `actionGroupExecutor` fields blank for 
                                                        #     this action group.
)

actionGroupId = response['agentActionGroup']['actionGroupId']

print("Waiting for action group status of 'ENABLED'...")

# Wait for action group to reach 'ENABLED' status
actionGroupStatus = ''
while actionGroupStatus != 'ENABLED':
    response = bedrock_agent.get_agent_action_group(
        agentId=agentId,
        actionGroupId=actionGroupId,
        agentVersion='DRAFT'
    )
    actionGroupStatus = response['agentActionGroup']['actionGroupState']
    print(f"Action Group status: {actionGroupStatus}")
    time.sleep(2)

print("Preparing the agent...")

# Prepare the agent for use
response = bedrock_agent.prepare_agent(
    agentId=agentId
)

print("Waiting for agent status of 'PREPARED'...")

# Wait for agent to reach 'PREPARED' status
agentStatus = ''
while agentStatus != 'PREPARED':
    response = bedrock_agent.get_agent(
        agentId=agentId
    )
    agentStatus = response['agent']['agentStatus']
    print(f"Agent status: {agentStatus}")
    time.sleep(2)

print("Creating an agent alias...")

# Create an alias for the agent
response = bedrock_agent.create_agent_alias(
    agentAliasName='test',
    agentId=agentId
)

agentAliasId = response['agentAlias']['agentAliasId']

# Wait for agent alias to be prepared
agentAliasStatus = ''
while agentAliasStatus != 'PREPARED':
    response = bedrock_agent.get_agent_alias(
        agentId=agentId,
        agentAliasId=agentAliasId
    )
    agentAliasStatus = response['agentAlias']['agentAliasStatus']
    print(f"Agent alias status: {agentAliasStatus}")
    time.sleep(2)

print('Done.\n')

print(f"agentId: {agentId}, agentAliasId: {agentAliasId}")

Phew! That was a lot, but we've successfully set up our Bedrock Agent. Let's break down what we did:

1. We created an IAM role and policy to allow our agent to invoke the foundation model.
2. We created the agent itself, using Claude 3 Sonnet as the foundation model.
3. We created an action group and enabled Code Interpreter.
4. We prepared the agent and created an alias for it.

Now that our agent is ready, let's set up the runtime client to interact with it!

In [ ]:
bedrock_agent_runtime = boto3.client(service_name = 'bedrock-agent-runtime', region_name = region_name)

## Step 5: Create an Invoke Function

Now, let's create a handy function to interact with our agent. This function will handle sending messages to the agent and processing its responses.

In [ ]:
def invoke(inputText, showTrace=False, endSession=False):

    try:

        # Invoke the Agent - Sends a prompt for the agent to process and respond to.
        response = bedrock_agent_runtime.invoke_agent(
            agentAliasId=agentAliasId,   # (string) – [REQUIRED] The alias of the agent to use.
            agentId=agentId,             # (string) – [REQUIRED] The unique identifier of the agent to use.
            sessionId=sessionId,         # (string) – [REQUIRED] The unique identifier of the session. Use the same value across requests to continue the same conversation.
            inputText=inputText,         # (string) - The prompt text to send the agent.
            endSession=endSession,       # (boolean) – Specifies whether to end the session with the agent or not.
            enableTrace=True,            # (boolean) – Specifies whether to turn on the trace or not to track the agent's reasoning process.
        )

        # To upload files to the agent, for use in the sandbox use sessionState:
            # sessionState = {
            #     "files": [
            #         {
            #             "name": "test_data.json",
            #             "source": {
            #                 "byteContent": {
            #                     "data": data,
            #                     "mediaType": "application/json"
            #                 },
            #                 "sourceType": "BYTE_CONTENT"
            #             },
            #             "useCase": "CODE_INTERPRETER"
            #         }
            #     ]
            # }

        # The response of this operation contains an EventStream member. 
        event_stream = response["completion"]

        # When iterated the EventStream will yield events.
        for event in event_stream:

            # chunk contains a part of an agent response
            if 'chunk' in event:
                chunk = event['chunk']
                if 'bytes' in chunk:
                    text = chunk['bytes'].decode('utf-8')
                    print(f"Chunk: {text}")
                else:
                    print("Chunk doesn't contain 'bytes'")

            # files contains intermediate response for code interpreter if any files have been generated.
            if 'files' in event:
                files = event['files']['files']
                for file in files:
                    name = file['name']
                    type = file['type']
                    bytes_data = file['bytes']
                    
                    # It the file is a PNG image then we can display it...
                    if type == 'image/png':
                        # Display PNG image using Matplotlib
                        img = plt.imread(io.BytesIO(bytes_data))
                        plt.figure(figsize=(10, 10))
                        plt.imshow(img)
                        plt.axis('off')
                        plt.title(name)
                        plt.show()
                        plt.close()
                        
                    # If the file is NOT a PNG then we save it to disk...
                    else:
                        # Save other file types to local disk
                        with open(name, 'wb') as f:
                            f.write(bytes_data)
                        print(f"File '{name}' saved to disk.")

    except Exception as e:
        print(f"Error: {e}")

This `invoke` function is our Swiss Army knife for interacting with the agent. It handles sending messages, processing responses, and even allows us to view and save files created by the agent.

## Step 6: Interacting with the Agent _(Remember this is preview.)_

In [ ]:
sessionId = str(uuid.uuid4())
invoke("Calculate the sum of the Fibonacci sequence up to 1000.")

In [ ]:
invoke("Plot a graph of the Fibonacci sequence.")

In [ ]:
sessionId = str(uuid.uuid4())

invoke("""Using the billing data provided below, create a bar 
graph that shows the total spend by product category (cat). 
The graph should have the category names on the x-axis and 
the total spend amount on the y-axis.

Billing Data: 
date,cat,id,product_name,cost 
2023-01-15,Electronics,E001,Smartphone,799.99 
2023-01-20,Home,H001,Vacuum Cleaner,199.99 
2023-02-03,Electronics,E002,Laptop,1299.99 
2023-02-10,Clothing,C001,Winter Jacket,129.99 
2023-02-25,Home,H002,Coffee Maker,89.99 
2023-03-05,Electronics,E003,Wireless Earbuds,159.99 
2023-03-12,Clothing,C002,Running Shoes,99.99 
2023-03-30,Home,H003,Blender,79.99 
2023-04-08,Electronics,E004,Smart Watch,299.99 
2023-04-15,Clothing,C003,Jeans,59.99 
2023-04-28,Home,H004,Toaster Oven,129.99 
2023-05-10,Electronics,E005,Tablet,499.99 
2023-05-18,Clothing,C004,Dress Shirt,49.99 
2023-05-25,Home,H005,Air Purifier,199.99 
2023-06-02,Electronics,E006,Gaming Console,399.99

Ensure that the graph is clearly labeled and easy to read. 
After generating the graph, provide a brief interpretation 
of the results, highlighting which category has the highest 
total spend and any other notable observations.""")

In [ ]:
invoke("Format the sales data into a JSON format. And save to a file.")

## Step 9: Cleaning Up

We've had a great time chatting with our Bedrock Agent, but now it's time to clean up. Let's delete the agent and its associated resources.

In [ ]:
response = bedrock_agent.delete_agent(
    agentId=agentId,
    skipResourceInUseCheck=True
)

response['agentStatus']

Finally, let's clean up the IAM role and policies we created for this demo.

In [ ]:
inline_policies = iam.list_role_policies(RoleName=role_name)
for policy_name in inline_policies.get('PolicyNames', []):
    iam.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
    print(f"Deleted inline policy: {policy_name}")

response = iam.delete_role(
    RoleName=role_name
)

print(f"Deleted role.")

## Conclusion

And there you have it! We've successfully created a Bedrock Agent, access to a sandbox to execute code in order to be able to answer complex queries, had a conversation with it, and then cleaned everything up. 

Through this process, we've seen how to:
1. Set up the necessary AWS resources for a Bedrock Agent
2. Create and configure an agent with Code Interpreter 
3. Interact with the agent and observe its responses
4. Clean up all the resources when we're done

This demo showcases the power of Bedrock Agents and how they can be used to create interactive AI assistants that can process complex queries. The possibilities for using this in various applications are endless!